In [26]:
import csv
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from collections import Counter

In [27]:
#Lectura de datos 
nodes=pd.read_csv("../Datos/musae_git_target.csv")
nodes = nodes.sort_values(by='id')
atributeCloseness = pd.read_csv("../Metricas/Closeness.csv")
atributeBetweenness = pd.read_csv("../Metricas/Betweenness.csv")
atributeDegree = pd.read_csv("../Metricas/Degree.csv")
atributeClustering = pd.read_csv("../Metricas/ClusteringC.csv")
atributeTriangles = pd.read_csv("../Metricas/Triangles.csv")
atributeSquares = pd.read_csv("../Metricas/Squares.csv")
atributeKCores = pd.read_csv("../Metricas/K_Core.csv")
atributeCommunities = pd.read_csv("../Metricas/Comunidades.csv")
atributesAsyn = pd.read_csv("../Metricas/Asyn_LPA.csv")

atributeCloseness = atributeCloseness.sort_values(by="id")
atributeBetweenness = atributeBetweenness.sort_values(by="id")
atributeDegree = atributeDegree.sort_values(by="id")
atributeClustering = atributeClustering.sort_values(by="id")
atributeTriangles = atributeTriangles.sort_values(by="id")
atributeSquares = atributeSquares.sort_values(by="id")
atributeKCores = atributeKCores.sort_values(by="id")
atributeCommunities = atributeCommunities.sort_values(by="id")
atributesAsyn = atributesAsyn.sort_values(by="id")

In [28]:
atributesAsyn.rename(columns={'Comunidad': 'asyn_lpa_community'}, inplace=True)

#Id a str 
nodes['id']= nodes['id'].astype(str)
atributeCloseness['id']= atributeCloseness['id'].astype(str)
atributeBetweenness['id']= atributeBetweenness['id'].astype(str)
atributeDegree['id']= atributeDegree['id'].astype(str)
atributeClustering['id']= atributeClustering['id'].astype(str)
atributeTriangles['id']= atributeTriangles['id'].astype(str)
atributeSquares['id']= atributeSquares['id'].astype(str)
atributeKCores['id']= atributeKCores['id'].astype(str)
atributeCommunities['id']= atributeCommunities['id'].astype(str)
atributesAsyn['id']= atributesAsyn['id'].astype(str)


In [29]:
#DataFrame
nodes = pd.merge(nodes, atributeCloseness, on='id', how='outer')
nodes = pd.merge(nodes, atributeBetweenness, on='id', how='outer')
nodes = pd.merge(nodes, atributeDegree, on='id', how='outer')
nodes = pd.merge(nodes, atributeClustering, on='id', how='outer')
nodes = pd.merge(nodes, atributeTriangles, on='id', how='outer')
nodes = pd.merge(nodes, atributeSquares, on='id', how='outer')
nodes = pd.merge(nodes, atributeKCores, on='id', how='outer')
nodes = pd.merge(nodes, atributeCommunities, on='id', how='outer')
nodes = pd.merge(nodes, atributesAsyn, on='id', how='outer')

In [30]:
nodes['id'] = pd.to_numeric(nodes['id'],errors ='coerce')
nodes = nodes.dropna(subset=['id'])
nodes['id'] = nodes['id'].astype(int)
nodes = nodes.sort_values(by='id', ascending=True)
nodes['id'] = nodes['id'].astype(str)
nodes.head(15)


,id,name,ml_target,Closeness_Centrality,Betweenness_Centrality,Degree_Centrality,Clustering_Coefficient,Triangles,Squares,K_Core,Comunidad,asyn_lpa_community
0,0,Eiryyy,0.0,0.275005,0.000000e+00,0.000027,0.000000,0,0.000000,1,0,0
1,1,shawflying,0.0,0.294956,1.149733e-06,0.000212,0.178571,5,0.072344,6,1,0
11112,2,JpMCarrilho,1.0,0.261845,0.000000e+00,0.000027,0.000000,0,0.000000,1,0,0
22223,3,SuhwanCha,0.0,0.278718,5.316292e-05,0.000133,0.000000,0,0.019178,4,2,0
31034,4,sunilangadi2,1.0,0.243084,6.134318e-09,0.000053,0.000000,0,0.000000,2,5,0
32145,5,j6montoya,0.0,0.343412,0.000000e+00,0.000027,0.000000,0,0.000000,1,0,0
33256,6,sfate,0.0,0.372244,2.098552e-06,0.000159,0.333333,5,0.038866,6,0,0
34367,7,amituuush,0.0,0.320201,6.164540e-07,0.000212,0.321429,9,0.054237,8,0,0
35478,8,mauroherlein,0.0,0.351534,1.784020e-07,0.000212,0.750000,21,0.057692,8,1,0
36589,9,ParadoxZero,0.0,0.343590,2.711870e-05,0.000186,0.238095,5,0.001971,5,2,0


In [31]:
columnsTN =[ 'Triangles','Comunidad', 'asyn_lpa_community']
scaler = MinMaxScaler()

nodes[columnsTN] = scaler.fit_transform(nodes[columnsTN])
nodes.head(15)

,id,name,ml_target,Closeness_Centrality,Betweenness_Centrality,Degree_Centrality,Clustering_Coefficient,Triangles,Squares,K_Core,Comunidad,asyn_lpa_community
0,0,Eiryyy,0.0,0.275005,0.000000e+00,0.000027,0.000000,0.000000,0.000000,1,0.000000,0.0
1,1,shawflying,0.0,0.294956,1.149733e-06,0.000212,0.178571,0.000062,0.072344,6,0.002227,0.0
11112,2,JpMCarrilho,1.0,0.261845,0.000000e+00,0.000027,0.000000,0.000000,0.000000,1,0.000000,0.0
22223,3,SuhwanCha,0.0,0.278718,5.316292e-05,0.000133,0.000000,0.000000,0.019178,4,0.004454,0.0
31034,4,sunilangadi2,1.0,0.243084,6.134318e-09,0.000053,0.000000,0.000000,0.000000,2,0.011136,0.0
32145,5,j6montoya,0.0,0.343412,0.000000e+00,0.000027,0.000000,0.000000,0.000000,1,0.000000,0.0
33256,6,sfate,0.0,0.372244,2.098552e-06,0.000159,0.333333,0.000062,0.038866,6,0.000000,0.0
34367,7,amituuush,0.0,0.320201,6.164540e-07,0.000212,0.321429,0.000112,0.054237,8,0.000000,0.0
35478,8,mauroherlein,0.0,0.351534,1.784020e-07,0.000212,0.750000,0.000262,0.057692,8,0.002227,0.0
36589,9,ParadoxZero,0.0,0.343590,2.711870e-05,0.000186,0.238095,0.000062,0.001971,5,0.004454,0.0


In [32]:
#Todos los argumentos 
tableAllAtributes = nodes[['id', 'name', 'ml_target','Closeness_Centrality', 'Betweenness_Centrality', 'Degree_Centrality', 'Clustering_Coefficient', 'Triangles', 'Squares', 'K_Core', 'Comunidad', 'asyn_lpa_community']]
tableAllAtributes.to_csv('../Tablas/TablaAtributos.csv', index=False)


,id,name,ml_target,Closeness_Centrality,Betweenness_Centrality,Degree_Centrality,Clustering_Coefficient,Triangles,Squares,K_Core,Comunidad,asyn_lpa_community
0,0,Eiryyy,0.0,0.275005,0.000000e+00,0.000027,0.000000,0.000000,0.000000,1,0.000000,0.000000
1,1,shawflying,0.0,0.294956,1.149733e-06,0.000212,0.178571,0.000062,0.072344,6,0.002227,0.000000
11112,2,JpMCarrilho,1.0,0.261845,0.000000e+00,0.000027,0.000000,0.000000,0.000000,1,0.000000,0.000000
22223,3,SuhwanCha,0.0,0.278718,5.316292e-05,0.000133,0.000000,0.000000,0.019178,4,0.004454,0.000000
31034,4,sunilangadi2,1.0,0.243084,6.134318e-09,0.000053,0.000000,0.000000,0.000000,2,0.011136,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
30774,37695,shawnwanderson,1.0,0.254356,0.000000e+00,0.000027,0.000000,0.000000,0.000000,1,0.819599,0.721003
30775,37696,kris-ipeh,0.0,0.287133,2.559261e-08,0.000053,0.000000,0.000000,0.005128,2,0.000000,0.000000
30776,37697,qpautrat,0.0,0.300441,1.357232e-07,0.000106,0.000000,0.000000,0.015656,4,0.002227,0.000000
30777,37698,Injabie3,1.0,0.262187,6.875323e-07,0.000080,0.000000,0.000000,0.005917,2,0.002227,0.000000


In [33]:
#Tabla sin Centralidad
tableNoCentral = nodes[['id','name', 'ml_target', 'Clustering_Coefficient', 'Triangles', 'Squares', 'K_Core', 'Comunidad', 'asyn_lpa_community']]
tableNoCentral.to_csv('../Tablas/TablaNoCentrality.csv', index=False)

In [34]:
#Tabla sin kernel 
tableNoKCore = nodes[['id','name', 'ml_target', 'Closeness_Centrality', 'Betweenness_Centrality', 'Degree_Centrality', 'Clustering_Coefficient', 'Triangles', 'Squares', 'Comunidad', 'asyn_lpa_community']]
tableNoKCore.to_csv('../Tablas/TablaNoKCore.csv', index=False)

In [35]:
#Tabla sin Comunidades
tableNoCommunities = nodes[['id','name', 'ml_target', 'Closeness_Centrality', 'Betweenness_Centrality', 'Degree_Centrality', 'Clustering_Coefficient', 'Triangles', 'Squares', 'K_Core', 'asyn_lpa_community']]
tableNoCommunities.to_csv('../Tablas/TablaNoCommunities.csv', index=False)

In [36]:
#Tabla sin agrupamiento
TablaNoClustering = nodes[['id','name', 'ml_target', 'Closeness_Centrality', 'Betweenness_Centrality', 'Degree_Centrality', 'Triangles', 'Squares', 'K_Core', 'Comunidad', 'asyn_lpa_community']]
TablaNoClustering.to_csv('../Tablas/TablaNoClustering.csv', index=False)